In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import re
from pathlib import Path
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
folder = '../data/ocred_pdf'

In [3]:
import os

In [17]:
paths = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

In [18]:
docs = []
for p in paths:
    with open(p, 'r', encoding='utf-8') as f:
        data = f.read()
    docs.append({
        'doc_name': os.path.splitext(os.path.basename(p))[0],
        'text': data
    })

In [19]:
df = pd.DataFrame(docs)
df

,doc_name,text
0,Онлайн_магистратура_«Машинное_обучение_и_анали...,× @СБЕР |\nСпециализированное высшее образован...
1,Особые_условия_для_поступления_в_Институт_№8_МАИ,Особые условия для поступления в Институт\n№8 ...
2,Постановление Правительства РФ от 27.04.2024 N...,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\nПОСТАНОВЛЕ...
3,Правила приема МАИ,Правила приема на обучение по образовательным ...
4,Правила Приема(Министерские),МИНИСТЕРСТВО юстиции РОССИЙСКОЙ ФЕТЕРАЦИИ\nЗАР...
5,Федеральный закон от 29 декабря 2012 г. N 273...,Федеральный закон от 29 декабря 2012 г. N 273-...


In [22]:
def chunk_text_sliding(text: str, window: int = 450, overlap: int = 50):
    words = text.split()
    step = window - overlap
    chunks = []
    for start in range(0, len(words), step):
        chunk = words[start:start + window]
        if not chunk:
            break
        chunks.append(" ".join(chunk))
    return chunks

In [23]:
records = []
for _, row in df.iterrows():
    chunks = chunk_text_sliding(
        row['text']
    )
    name = row['doc_name']

    for i, chunk in enumerate(chunks, 1):
        records.append({
            'doc_name':  row['doc_name'],
            'chunk_id':  i,
            'chunk_text': chunk,
            'text_name_source': f"[{name}] {chunk}"
        })

chunks_df = pd.DataFrame(records)
chunks_df.head()

,doc_name,chunk_id,chunk_text,text_name_source
0,Онлайн_магистратура_«Машинное_обучение_и_анали...,1,× @СБЕР | Специализированное высшее образовани...,[Онлайн_магистратура_«Машинное_обучение_и_анал...
1,Особые_условия_для_поступления_в_Институт_№8_МАИ,1,Особые условия для поступления в Институт №8 М...,[Особые_условия_для_поступления_в_Институт_№8_...
2,Особые_условия_для_поступления_в_Институт_№8_МАИ,2,МАИ -- это управление жизненным циклом перспек...,[Особые_условия_для_поступления_в_Институт_№8_...
3,Постановление Правительства РФ от 27.04.2024 N...,1,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕН...,[Постановление Правительства РФ от 27.04.2024 ...
4,Постановление Правительства РФ от 27.04.2024 N...,2,Федерации М. МИШУСТИН УТВЕРЖДЕНО постановление...,[Постановление Правительства РФ от 27.04.2024 ...


In [24]:
chunks_df['dialog_plain'] = chunks_df['text_name_source']

In [25]:
chunks_df.to_csv('pdf_df.csv')

In [26]:
folder = '../data/vk_parse'

In [35]:
import os
import re

def process_vk_files(folder_path: str):

    docs = []
    
    paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
             if os.path.isfile(os.path.join(folder_path, f))]
    
    for p in paths:
        try:
            with open(p, 'r', encoding='utf-8') as f:
                data = f.read()
            
            chunks = re.split(r'\$\$\$\d+', data)

            for chunk in chunks:
                clean_chunk = ' '.join(chunk.strip().split())
                
                if len(clean_chunk.split()) >= 30:
                    base_name = os.path.splitext(os.path.basename(p))[0]
                    docs.append({
                        'doc_name': base_name,
                        'text': clean_chunk, 
                        'dialog_plain': f"[{base_name}] {clean_chunk}"
                    })
                    
        except Exception as e:
            print(f"Ошибка при обработке файла {p}: {e}")
    
    return docs

In [38]:
folder = '../data/vk_parse'
processed_docs = process_vk_files(folder)

In [39]:
data = pd.DataFrame(processed_docs)
data

,doc_name,text,dialog_plain
0,8 факультет МАИ,1733997608 Твой Альфа-Шанс на 300 000 ₽ ⚡️ Дар...,[8 факультет МАИ] 1733997608 Твой Альфа-Шанс н...
1,IT-Центр МАИ,"1745219444 Какие планы на лето? Сессия, практи...",[IT-Центр МАИ] 1745219444 Какие планы на лето?...
2,Институт № 5 МАИ,1744829478 ❗❗❗ВНИМАНИЕ❗❗❗ Начинается прием зая...,[Институт № 5 МАИ] 1744829478 ❗❗❗ВНИМАНИЕ❗❗❗ Н...
3,Институт №1 Авиационная Техника МАИ,"1745861590 $$$ 1745827600 Приветствуем тебя, ч...",[Институт №1 Авиационная Техника МАИ] 17458615...
4,ИНСТИТУТ №2 МАИ,1745066468 $$$ 1744965351 Начинается прием зая...,[ИНСТИТУТ №2 МАИ] 1745066468 $$$ 1744965351 На...
5,МАИ Московский авиационный институт,1742448600 МАИ — навсегда в твоём сердце За 95...,[МАИ Московский авиационный институт] 1742448...
6,МАИ Институт №9 Общеинженерной подготовки,1745227213 Начинается прием заявлений на матер...,[МАИ Институт №9 Общеинженерной подготовки] 1...
7,Поступи в Институт №7 МАИ,1740983759 $$$ 1738313017 $$$ 1736491987 $$$ 1...,[Поступи в Институт №7 МАИ] 1740983759 $$$ 173...
8,ФРЭЛА МАИ,"1744356960 передайте своему товарищу - может, ...",[ФРЭЛА МАИ] 1744356960 передайте своему товари...


In [ ]:
output_folder = '../data/processed_vk'
os.makedirs(output_folder, exist_ok=True)
df.to_csv(os.path.join(output_folder, 'vk_posts_processed.csv'), index=False)

In [ ]:


# Конвертируем в DataFrame
df = pd.DataFrame(processed_docs)
print(df.head())

# Сохраняем результат
output_folder = '../data/processed_vk'
os.makedirs(output_folder, exist_ok=True)
df.to_csv(os.path.join(output_folder, 'vk_posts_processed.csv'), index=False)

In [32]:
import re

def process_vk_posts(posts: list[str]) -> pd.DataFrame:
    processed_data = []
    
    for post in posts:
        chunks = re.split(r'\$\$\$\d*', post)
        
        for chunk in chunks:
            clean_chunk = ' '.join(chunk.strip().split())
            
            if len(clean_chunk.split()) >= 30:
                processed_data.append({
                    'original_post': post,
                    'processed_chunk': clean_chunk
                })
    
    df = pd.DataFrame(processed_data)
    
    return df

In [33]:
df

,doc_name,text
0,Онлайн_магистратура_«Машинное_обучение_и_анали...,× @СБЕР |\nСпециализированное высшее образован...
1,Особые_условия_для_поступления_в_Институт_№8_МАИ,Особые условия для поступления в Институт\n№8 ...
2,Постановление Правительства РФ от 27.04.2024 N...,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ\nПОСТАНОВЛЕ...
3,Правила приема МАИ,Правила приема на обучение по образовательным ...
4,Правила Приема(Министерские),МИНИСТЕРСТВО юстиции РОССИЙСКОЙ ФЕТЕРАЦИИ\nЗАР...
5,Федеральный закон от 29 декабря 2012 г. N 273...,Федеральный закон от 29 декабря 2012 г. N 273-...


In [30]:
docs = []
for p in paths:
    with open(p, 'r', encoding='utf-8') as f:
        data = f.read()
    docs.append({
        'doc_name': os.path.splitext(os.path.basename(p))[0],
        'text': data, 
        'dialog_plain': f"[{os.path.splitext(os.path.basename(p))[0]}] {data}"
    })

In [31]:
data = pd.DataFrame(docs)
data

,doc_name,text,dialog_plain
0,8 факультет МАИ,1733997608\nТвой Альфа-Шанс на 300 000 ₽ ⚡️ \n...,[8 факультет МАИ] 1733997608\nТвой Альфа-Шанс ...
1,IT-Центр МАИ,"1745219444\nКакие планы на лето? Сессия, практ...",[IT-Центр МАИ] 1745219444\nКакие планы на лето...
2,Институт № 5 МАИ,1744829478\n❗❗❗ВНИМАНИЕ❗❗❗ \nНачинается прием ...,[Институт № 5 МАИ] 1744829478\n❗❗❗ВНИМАНИЕ❗❗❗ ...
3,Институт №1 Авиационная Техника МАИ,1745861590\n\n$$$\n1745827600\nПриветствуем те...,[Институт №1 Авиационная Техника МАИ] 17458615...
4,ИНСТИТУТ №2 МАИ,1745066468\n\n$$$\n1744965351\nНачинается прие...,[ИНСТИТУТ №2 МАИ] 1745066468\n\n$$$\n174496535...
5,МАИ Московский авиационный институт,1742448600\nМАИ — навсегда в твоём сердце \n\n...,[МАИ Московский авиационный институт] 1742448...
6,МАИ Институт №9 Общеинженерной подготовки,1745227213\nНачинается прием заявлений на мате...,[МАИ Институт №9 Общеинженерной подготовки] 1...
7,Поступи в Институт №7 МАИ,1740983759\n\n$$$\n1738313017\n\n$$$\n17364919...,[Поступи в Институт №7 МАИ] 1740983759\n\n$$$\...
8,ФРЭЛА МАИ,"1744356960\nпередайте своему товарищу - может,...",[ФРЭЛА МАИ] 1744356960\nпередайте своему товар...


In [4]:
folder = '../data'

In [5]:
paths = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
paths

['../data\\2025-04-29_13-41-54_parsed.txt']

In [9]:
with open('../data/2025-04-29_13-41-54_parsed.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [12]:
pages = re.split(r'--- Page: (.+?) ---', text)[1:]

data = []

for i in range(0, len(pages), 2):
    url = pages[i].strip()
    content = pages[i+1].strip()
    
    cleaned_lines = []
    for line in content.split('\n'):
        line = line.strip()
        
        line = re.sub(r'^\[.*?\]', '', line).strip()
        
        if re.match(r'^[A-Za-zА-Яа-я]:', line):
            line = re.sub(r'^[A-Za-zА-Яа-я]:', '', line).strip()

        line = re.sub(r'^[\'"]', '', line)
        line = re.sub(r'[\'"]$', '', line).strip()
        
        if line:  
            cleaned_lines.append(line)
    
    page_content = '\n'.join(cleaned_lines)
    
    data.append({'url': url, 'content': page_content})


df = pd.DataFrame(data)


In [13]:
df

,url,content
0,https://priem.mai.ru/,Приёмная комиссия МАИ\nБаннеры\nДниоткрытыхдве...
1,https://priem.mai.ru/calendar/,Календарь абитуриента\nКалендарьабитуриента\nД...
2,https://priem.mai.ru/results/orders/2024.php,Приказы о зачислении 2024\nПриказы о зачислени...
3,https://priem.mai.ru/contacts/,Контакты\nКонтакты\nОсновной кампус\nКак пройт...
4,https://priem.mai.ru/news/item.php?id=184677,МАИ проведёт День открытых дверей для поступаю...
...,...,...
733,https://priem.mai.ru/articles/?PAGEN_1=25,Статьи\nВ МАИ определили победителей школьной ...
734,https://priem.mai.ru/articles/?PAGEN_1=26,Статьи\nУченик Предуниверсария МАИ создал Tele...
735,https://priem.mai.ru/articles/?PAGEN_1=27,Статьи\nИз первокурсника в суперинженера: особ...
736,https://priem.mai.ru/articles/?PAGEN_1=28,Статьи\nМАИ привлёк школьников к дистанционной...


In [14]:
df.to_csv('../data/support/pages_mai.csv', index=False, encoding='utf-8-sig')